## Функции шифрования

### Цезарь

In [1]:
class CaesarEncryptor:
    def __init__(self, alphabet):
        self.alphabet = alphabet

    def encrypt(self, ptext, key):
        if type(key) == int:
            return self.__encrypt_int(ptext, key)
        elif type(key) == str:
            return self.__encrypt_char(ptext, key)
        else:
            raise Exception("Key should be integer shift or a single character in alphabet")
            
    def decrypt(self, ctext, key):
        if type(key) == int:
            return self.__encrypt_int(ctext, -key)
        elif type(key) == str:
            return self.__decrypt_char(ctext, key)
        else:
            raise Exception("Key should be integer shift or a single character in alphabet")
    
    def __encrypt_int(self, ptext, key):
        while key < 0:
            key += len(self.alphabet)
            
        ctext = []
        
        for c in ptext:
            pos = alphabet.find(c)
            if pos == -1:
                raise Exception("Text not in alphabet")
            ctext.append(self.alphabet[(pos + key) % len(self.alphabet)])
            
        return "".join(ctext)
    
    def __encrypt_char(self, ptext, key):
        shift = self.__char_to_shift(key)
        return self.__encrypt_int(ptext, shift)
    
    def __decrypt_char(self, ctext, key):
        shift = -self.__char_to_shift(key)
        return self.__encrypt_int(ctext, shift)
    
    def __char_to_shift(self, key):
        if len(key) != 1:
            raise Exception("Key not a single character")
        shift = self.alphabet.find(key)
        if shift == -1:
            raise Exception("Key not in alphabet")
        return shift

#### Проверка

In [2]:
alphabet = "abcdefghijklmnopqrstuvwxyz"

encryptor = CaesarEncryptor(alphabet)

ptext = "kekcheburek"

ctext = encryptor.encrypt(ptext, 1)
assert ptext == encryptor.decrypt(ctext, 1)

ctext = encryptor.encrypt(ptext, "z")
assert ptext == encryptor.decrypt(ctext, "z")

### Виженер

In [3]:
class LosungEncryptor:
    def __init__(self, alphabet):
        self.caesar = CaesarEncryptor(alphabet)
    
    def encrypt(self, ptext, key):
        if type(key) != str and type(key) != list:
            raise Exception("Key should be string or list of integer")
        if len(key) == 0:
            raise Exception("Zero key")
            
        subtexts = []
        
        for i in range(len(key)):
            subtexts.append(self.caesar.encrypt(ptext[i::len(key)], key[i]))
            
        ctext = []
        for i in range(len(subtexts[0])):
            for st in subtexts:
                if i < len(st):
                    ctext.append(st[i])
        return "".join(ctext)
    
    def decrypt(self, ctext, key):
        if type(key) != str and type(key) != list:
            raise Exception("Key should be string or list of integer")
        if len(key) == 0:
            raise Exception("Zero key")
            
        subtexts = []
        
        for i in range(len(key)):
            subtexts.append(self.caesar.decrypt(ctext[i::len(key)], key[i]))
            
        ptext = []
        for i in range(len(subtexts[0])):
            for st in subtexts:
                if i < len(st):
                    ptext.append(st[i])
        return "".join(ptext)

#### Проверка

In [4]:
alphabet = "abcdefghijklmnopqrstuvwxyz"

encryptor = LosungEncryptor(alphabet)

ptext = "kekcheburek"

ctext = encryptor.encrypt(ptext, [1, 5, 12, 3])
assert ptext == encryptor.decrypt(ctext, [1, 5, 12, 3])

ctext = encryptor.encrypt(ptext, "losung")
assert ptext == encryptor.decrypt(ctext, "losung")

In [5]:
ctext = 'beeakfydjxuqyhyjiqryhtyjiqfbqduyjiikfuhcqd'
encryptor = CaesarEncryptor(alphabet)

encryptor.decrypt(ctext, -10)

'lookupintheairitsabirditsaplaneitssuperman'

## Функции анализа

In [6]:
from operator import itemgetter

class CaesarAnalyzer:
    def __init__(self, alphabet, probs):
        self.alphabet = alphabet
        self.probs = probs
        self.caesar = CaesarEncryptor(alphabet)
        
        for i in alphabet:
            if not i in probs:
                raise Exception(f"Character {i} has no probability")
    
    def calc_index(self, text):
        summ = 0
        counts = self.count_letters(text)
        
        for i in alphabet:
            summ += (counts[i]/ len(text))**2
            
        return summ
    
    def calc_key_index(self, text):
        summ = 0
        counts = self.count_letters(text)
        
        for i in alphabet:
            summ += counts[i] / len(text) * probs[i]
            
        return summ
    
    def count_letters(self, text):
        counts = dict()
        
        for i in alphabet:
            counts[i] = 0
            
        for i in text:
            counts[i] += 1
            
        return counts
    
    def brute_keys(self, ctext):
        indexes = dict()
        
        for i in alphabet:
            indexes[i] = self.calc_key_index(self.caesar.decrypt(ctext, i))
            
        return indexes
    
    
    def top_keys(self, ctext):
        return [i[0] for i in sorted(self.brute_keys(ctext).items(), key = itemgetter(1), reverse=True)]

In [7]:
class LosungAnalyser:
    def __init__(self, alphabet, probs):
        self.caesar = CaesarAnalyzer(alphabet, probs)
        self.alphabet = alphabet
        self.probs = probs
        
    def brute_lengths(self, ctext, maxlen):
        lens = dict()
        lens[0] = [self.caesar.calc_index(ctext)]
        for i in range(2, maxlen + 1):
            cur_indexes = []
            
            for j in range(i):
                cur_indexes.append(self.caesar.calc_index(ctext[j::i]))
           
            lens[i] = cur_indexes
            
        return lens
    
    def top_lengths(self, ctext, maxlen):
        lens_ids = self.brute_lengths(ctext, maxlen)
        
        for i in lens_ids.keys():
            lens_ids[i] = sum(lens_ids[i]) / len(lens_ids[i])
            
        return [i[0] for i in sorted(lens_ids.items(), key = itemgetter(1), reverse=True)]
    
    def top_keys(self, ctext, length):
        top = []
        
        for i in range(length):
            top.append(self.caesar.top_keys(ctext[i::length]))
            
        return top
        

In [8]:
probs = {
    'a':0.0817,
    'b':0.0149,
    'c':0.0278,
    'd':0.0425,
    'e':0.127,
    'f':0.0223,
    'g':0.0202,
    'h':0.0609,
    'i':0.0697,
    'j':0.0015,
    'k':0.0077,
    'l':0.0403,
    'm':0.0241,
    'n':0.0675,
    'o':0.0751,
    'p':0.0193,
    'q':0.001,
    'r':0.0599,
    's':0.0633,
    't':0.0906,
    'u':0.0276,
    'v':0.0098,
    'w':0.0236,
    'x':0.0015,
    'y':0.0197,
    'z':0.0005,
}

print(sum([i for i in probs.values()]))

[i for i in sorted(probs.items(), key = itemgetter(1), reverse=True)]

0.9999999999999999


[('e', 0.127),
 ('t', 0.0906),
 ('a', 0.0817),
 ('o', 0.0751),
 ('i', 0.0697),
 ('n', 0.0675),
 ('s', 0.0633),
 ('h', 0.0609),
 ('r', 0.0599),
 ('d', 0.0425),
 ('l', 0.0403),
 ('c', 0.0278),
 ('u', 0.0276),
 ('m', 0.0241),
 ('w', 0.0236),
 ('f', 0.0223),
 ('g', 0.0202),
 ('y', 0.0197),
 ('p', 0.0193),
 ('b', 0.0149),
 ('v', 0.0098),
 ('k', 0.0077),
 ('j', 0.0015),
 ('x', 0.0015),
 ('q', 0.001),
 ('z', 0.0005)]

In [9]:
analyzer = LosungAnalyser(alphabet, probs)

In [10]:
ctext = "CHREEVOAHMAERATBIAXXWTNXBEEOPHBSBQMQEQERBW\
RVXUOAKXAOSXXWEAHBWGJMMQMNKGRFVGXWTRZXWIAK\
LXFPSKAUTEMNDCMGTSXMXBTUIADNGMGPSRELXNJELX\
VRVPRTULHDNQWTWDTYGBPHXTFALJHASVBFXNGLLCHR\
ZBWELEKMSJIKNBHWRJGNMGJSGLXFEYPHAGNRBIEQJT\
AMRVLCRREMNDGLXRRIMGNSNRWCHRQHAEYEVTAQEBBI\
PEEWEVKAKOEWADREMXMTBHHCHRTKDNVRZCHRCLQOHP\
WQAIIWXNRMGWOIIFKEE".lower()

In [11]:
analyzer.top_lengths(ctext, 15)

[15, 10, 13, 5, 14, 12, 11, 9, 8, 7, 6, 3, 4, 2, 0]

In [15]:
for i in analyzer.top_keys(ctext, 5):
    print(i)

['j', 'w', 'i', 'p', 'x', 't', 's', 'y', 'm', 'f', 'k', 'n', 'o', 'd', 'u', 'q', 'z', 'c', 'e', 'a', 'v', 'l', 'b', 'r', 'g', 'h']
['a', 'z', 'o', 'l', 'n', 'e', 'b', 'w', 'p', 'k', 'm', 't', 'q', 'g', 'd', 'f', 's', 'v', 'h', 'c', 'u', 'r', 'x', 'j', 'i', 'y']
['n', 'j', 'a', 'y', 'r', 'e', 'd', 'c', 'z', 'p', 'g', 'w', 'q', 'o', 'x', 'h', 'k', 't', 'm', 'u', 'v', 'f', 'i', 'l', 'b', 's']
['e', 't', 'i', 'a', 'r', 'x', 'l', 'q', 'm', 'w', 'y', 'd', 's', 'p', 'z', 'h', 'f', 'g', 'u', 'v', 'k', 'j', 'n', 'c', 'b', 'o']
['t', 'x', 'e', 'p', 's', 'd', 'g', 'i', 'h', 'z', 'j', 'f', 'n', 'u', 'o', 'm', 'y', 'c', 'a', 'k', 'l', 'q', 'b', 'w', 'r', 'v']


In [13]:
encryptor = LosungEncryptor(alphabet)

In [14]:
encryptor.decrypt(ctext, 'janet')

'thealmondtreewasintentativeblossomthedayswerelongeroftenendingwithmagnificenteveningsofcorrugatedpinkskiesthehuntingseasonwasoverwithhoundsandgunsputawayforsixmonthsthevineyardswerebusyagainasthewellorganizedfarmerstreatedtheirvinesandthemorelackadaisicalneighborshurriedtodothepruningtheyshouldhavedoneinnovember'

# Подстановки

In [16]:
import copy
import random
import pandas as pd

def generate_alphabet():
    alphabet = list()
    for i in range(256):
        alphabet.append(chr(i))
    return alphabet

def generate_cipher(alphabet):
    temp = copy.copy(alphabet)
    cipher = list()
    while len(temp):
        i = random.randrange(0, len(temp))
        cipher.append(temp[i])
        temp.remove(temp[i])
    return cipher

def encrypt(alphabet, cipher, text):
    cipher_text = ""
    for c in text:
        cipher_text += str(cipher[alphabet.index(c)])
    return cipher_text

def unique(lst):
    u = list()
    for c in lst:
        if not c in u:
            u.append(c)
    return u

def calculate_freqs(alph, text, cipher):
    counts = []
    freqs = []
    for c in cipher:
        counts.append(text.count(c))
        freqs.append(counts[-1]/len(text))
    df = pd.DataFrame({"cipher" : cipher, "alph" : alph, "count" : counts, 'freq' : freqs,})
    df = df.sort_values(["count"], ascending = False)
    df = df[df['count'] != 0]
    return df

def get_n_grams(text, n=2, included = True):
    n_grams = []
    gramm = ''
    for i in text:
        gramm = gramm + i
        if (len(gramm) > n):
            gramm = gramm[1:]
            n_grams.append(gramm)
            if not included:
                for i in range(1, 15):
                    founded = False
                    n_1_grams = get_n_grams(text, n+i)
                    for g in n_1_grams:
                        if text.count(g) > 1 and g.count(gramm):
                            founded = True
                            break
                    if (founded):
                        n_grams.remove(gramm)
                        break
                
    return unique(n_grams)

def count_n_grams(ctext, text):
    counts = list()
    sources = []

    grs = []
    for i in range(2, 3):
        grs += get_n_grams(text, i, True)

    for gr in grs:
        counts.append(text.count(gr))
        ind = text.index(gr)
        sources.append(ctext[ind:ind + len(gr)])

    df = pd.DataFrame({"gramm" : grs, "count" : counts, 'source' : sources,})
    df = df.sort_values(["count"], ascending = False)
    df = df[df['count'] > 1]
    return df

#### Проверка

In [17]:
ctext = ('YIFQMZRWQFYVECFMDZPCVMRZWNMDZVEJBTXCDDUMJ\n\
NDIFEFMDZCDMQZKCEYFCJMYRNCWJCSZREXCHZUNMXZ\n\
NZUCDRJXYYSMRTMEYIFZWDYVZVYFZUMRZCRWNZDZJJ\n\
XZWGCHSMRNMDHNCMFQCHZJMXJZWIEJYUCFWDJNZDIR'.lower().replace(' ', ''))

ctext = ('YIFQMZRWQFYVECFMDZPCVMRZWNMDZVEJBTXCDDUMJ'.lower().replace(' ', ''))

ctext = 'EMGLOSUDCGDNCUSWYSFHNSFCYKDPUMLWGYICOXYSIPJCK\
QPKUGKMGOLICGINCGACKSNISACYKZSCKXECJCKSHYSXCG\
OIDPKZCNKSHICGIWYGKKGKGOLDSILKGOIUSIGLEDSPWZU\
GFZCCNDGYYSFUSZCNXEOJNCGYEOWEUPXEZGACGNFGLKNS\
ACIGOIYCKXCJUCIUZCFZCCNDGYYSFEUEKUZCSOCFZCCNC\
IACZEJNCSHFZEJZEGMXCYHCJUMGKUCY'.lower()

alphabet = list('abcdefghijklmnopqrstuvwxyz')

In [19]:
alph   = unique(list('fcgznys'.lower()) + alphabet)
cipher = unique(list('wethlon'.upper()) + alphabet)

alph = unique(alph + cipher)
cipher = unique(cipher + alph)

print(alph)
print(cipher)

ptext = encrypt(alph, cipher, ctext)
calculate_freqs(alph, ptext, cipher)

['f', 'c', 'g', 'z', 'n', 'y', 's', 'a', 'b', 'd', 'e', 'h', 'i', 'j', 'k', 'l', 'm', 'o', 'p', 'q', 'r', 't', 'u', 'v', 'w', 'x', 'W', 'E', 'T', 'H', 'L', 'O', 'N']
['W', 'E', 'T', 'H', 'L', 'O', 'N', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


,cipher,alph,count,freq
1,E,c,37,0.144531
2,T,g,24,0.093750
6,N,s,20,0.078125
14,h,k,18,0.070312
5,O,y,15,0.058594
12,f,i,15,0.058594
22,p,u,14,0.054688
3,H,z,13,0.050781
4,L,n,13,0.050781
10,d,e,12,0.046875


In [20]:
print(ptext)

djTikNpcETcLEpNrONWeLNWEOhclpjirTOfEksONflgEhmlhpThjTkifETfLETaEhNLfNaEOhHNEhsdEgEhNeONsETkfclhHELhNefETfrOThhThTkicNfihTkfpNfTidcNlrHpTWHEELcTOONWpNHELsdkgLETOdkrdplsdHTaETLWTihLNaEfTkfOEhsEgpEfpHEWHEELcTOONWdpdhpHENkEWHEELEfaEHdgLENeWHdgHdTjsEOeEgpjThpEO


In [21]:
count_n_grams(ctext, ptext)

,gramm,count,source
7,ET,7,cg
73,HE,7,zc
74,EL,5,cn
10,LE,5,nc
15,ON,5,ys
40,Eh,5,ck
54,aE,5,ac
48,Tk,5,go
30,TO,4,gy
21,EO,4,cy


In [22]:
ctext = 'KQEREJEBCPPCJCRKIEACUZBKRVPKRBCIBQCARBJCVFCUP\
KRIOFKPACUZQEPBKRXPEIIEABDKPBCPFCDCCAFIEABDKP\
BCPFEQPKAZBKRHAIBKAPCCIBURCCDKDCCJCIDFUIXPAFF\
ERBICZDFKABICBBENEFCUPJCVKABPCYDCCDPKBCOCPERK\
IVKSCPICBRKIJPKABI'.lower()

In [24]:
alph   = unique(list(''.lower()) + alphabet)
cipher = unique(list(''.upper()) + alphabet)

alph = unique(alph + cipher)
cipher = unique(cipher + alph)

print(alph)
print(cipher)

ptext = encrypt(alph, cipher, ctext)
calculate_freqs(alph, ptext, cipher)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


,cipher,alph,count,freq
2,c,c,32,0.161616
1,b,b,21,0.106061
15,p,p,20,0.101010
10,k,k,20,0.101010
8,i,i,16,0.080808
0,a,a,13,0.065657
4,e,e,13,0.065657
17,r,r,12,0.060606
5,f,f,10,0.050505
3,d,d,9,0.045455


In [25]:
import gmpy2

class AffineEncryptor:
    def __init__(self, alphabet):
        self.alphabet = alphabet

    def encrypt(self, ptext, factor, shift):
        return self.__encrypt(ptext, factor, shift)
            
    def decrypt(self, ctext, factor, shift):
        return self.__encrypt(ctext, gmpy2.invert(factor, len(self.alphabet)), -shift)
    
    def __encrypt(self, ptext, factor, shift):
        while shift < 0:
            shift += len(self.alphabet)
            
        ctext = []
        
        for c in ptext:
            pos = alphabet.find(c)
            if pos == -1:
                raise Exception("Text not in alphabet")
            ctext.append(self.alphabet[(pos * factor + shift) % len(self.alphabet)])
            
        return "".join(ctext)

ModuleNotFoundError: No module named 'gmpy2'

In [23]:
from operator import itemgetter

class AffineAnalyzer:
    def __init__(self, alphabet, probs):
        self.alphabet = alphabet
        self.probs = probs
        self.affine = AffineEncryptor(alphabet)
        
        for i in alphabet:
            if not i in probs:
                raise Exception(f"Character {i} has no probability")
    
    def calc_index(self, text):
        summ = 0
        counts = self.count_letters(text)
        
        for i in alphabet:
            summ += (counts[i]/ len(text))**2
            
        return summ
    
    def calc_key_index(self, text):
        summ = 0
        counts = self.count_letters(text)
        
        for i in alphabet:
            summ += counts[i] / len(text) * probs[i]
            
        return summ
    
    def count_letters(self, text):
        counts = dict()
        
        for i in alphabet:
            counts[i] = 0
            
        for i in text:
            counts[i] += 1
            
        return counts
    
    def brute_keys(self, ctext):
        indexes = dict()
        
        for i in range(len(alphabet)):
            for j in range(len(alphabet)):
                try:
                    indexes[(i, j)] = self.calc_key_index(self.affine.decrypt(ctext, i, j))
                except Exception:
                    pass
            
        return indexes
    
    
    def top_keys(self, ctext):
        return [i[0] for i in sorted(self.brute_keys(ctext).items(), key = itemgetter(1), reverse=True)]

In [24]:
alphabet = "abcdefghijklmnopqrstuvwxyz"

encryptor = AffineEncryptor(alphabet)

ptext = "kekcheburek"

ctext = encryptor.encrypt(ptext, 1, 1)
assert ptext == encryptor.decrypt(ctext, 1, 1)
